# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [17]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [18]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [19]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [20]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [21]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [22]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [23]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [24]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [25]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [26]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [27]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [28]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [29]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

2025-09-02 09:16:32,088 - 139797554670464 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [30]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [31]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for AAPL in order to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for AAPL. Utilize all available data sources and analytical tools."}

> Entering new CrewAgentExecutor chain...
I should start by reading the latest news and updates on AAPL to gather insights that may impact the stock price.

Action: Read website content
Action Input: {"website_url": "https://www.marketwatc

Final Answer: Unfortunately, the website link provided to access information on statistical modeling and machine learning techniques used to identify the upward trend in AAPL's performance is not available. It seems like the specific details on this topic cannot be retrieved at the moment.

> Finished chain.
 

Unfortunately, the website link provided to access information on statistical modeling and machine learning techniques used to identify the upward trend in AAPL's performance is not available. It seems like the specific details on this topic cannot be retrieved at the moment.

Final Answer: 
Based on the insights provided by the Data Analyst and the recommendation from the Trading Strategy Developer, the team has developed a swing trading strategy for AAPL. This strategy involves buying AAPL stock during short-term dips and selling during short-term peaks to capitalize on the identified upward trend in AAPL's performance over the past quarter. The strategy aligns with the user's

I need to gather detailed information on potential risks associated with trading strategies and execution plans for AAPL, as well as identify mitigation strategies to address these risks.

Action: Read website content
Action Input: {"website_url": "https://www.apple.com/investor/"} 

Just a moment...Enable JavaScript and cookies to continue

Thought: I need to try a different approach to gather information on potential risks associated with trading strategies and execution plans for AAPL.

Action: Search the internet
Action Input: {"search_query": "AAPL trading risks and execution plans"} 


Search results: Title: These Are the Catalysts and Risks that Will Move Apple ...
Link: https://www.tipranks.com/news/these-are-the-catalysts-and-risks-that-will-move-apple-stock-aapl
Snippet: Turning to Wall Street, analysts have a Moderate Buy consensus rating on AAPL stock based on 15 Buys, 11 Holds, and one Sell assigned in the ...
---
Title: Analyst Explains Apple's (AAPL) 'Biggest Risk' – 'Si

- Display the final result as Markdown.

In [32]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL Trading Strategies and Execution Plans:

After conducting a comprehensive risk analysis for the proposed trading strategies and execution plans for AAPL, the following high-impact risks have been identified along with recommended mitigation strategies:

1. Market Volatility Risk:
- Potential impact: Market volatility can lead to significant fluctuations in stock prices, affecting the profitability of trades.
- Mitigation strategy: Implement stop-loss orders and diversify the portfolio to reduce exposure to market volatility.

2. Regulatory Risk:
- Potential impact: Changes in regulations or government policies can have a direct impact on AAPL's trading activities.
- Mitigation strategy: Stay informed about regulatory changes and adapt trading strategies accordingly to comply with regulations.

3. Technology Risk:
- Potential impact: Technological failures or disruptions can hinder the execution of trades and result in financial losses.
- Mitigation strategy: Implement backup systems and regularly update technology infrastructure to minimize the risk of technical failures.

By effectively managing these high-impact risks through the recommended mitigation strategies, AAPL can navigate potential challenges and optimize its trading strategies for maximum profitability.